In [ ]:
import os
os.environ["OPENAI_API_KEY"]="OPENAI_API_KEY"

In [7]:
!pip install chromadb

In [8]:
!pip install langchain

Importing Necessary Libraries


In [9]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import chroma

Loading and splitting the text file


In [10]:
from langchain.text_splitter import CharacterTextSplitter

# Open and read the text file
file_path = r"D:\data science QT\notes\DataSets\Alice's Adventures in Wonderland.txt"  # Raw string for file path
full_text = open(file_path, "r", encoding="utf-8").read()  # Specify encoding to handle special characters

# Initialize the text splitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# Split the text into chunks
texts = text_splitter.split_text(full_text)

# Output
print(f"Number of chunks: {len(texts)}")
for i, chunk in enumerate(texts[:5]):  # Print the first 5 chunks
    print(f"Chunk {i+1}:\n{chunk}\n")


Created a chunk of size 1216, which is longer than the specified 1000


Number of chunks: 183
Chunk 1:
Alice's Adventures in Wonderland

                ALICE'S ADVENTURES IN WONDERLAND

                          Lewis Carroll

               THE MILLENNIUM FULCRUM EDITION 3.0


                            CHAPTER I

                      Down the Rabbit-Hole


  Alice was beginning to get very tired of sitting by her sister
on the bank, and of having nothing to do:  once or twice she had
peeped into the book her sister was reading, but it had no
pictures or conversations in it, `and what is the use of a book,'
thought Alice `without pictures or conversation?'

  So she was considering in her own mind (as well as she could,
for the hot day made her feel very sleepy and stupid), whether
the pleasure of making a daisy-chain would be worth the trouble
of getting up and picking the daisies, when suddenly a White
Rabbit with pink eyes ran close by her.

Chunk 2:
There was nothing so VERY remarkable in that; nor did Alice
think it so VERY much out of the way to 

Embedding the text and storing in chroma


In [11]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter

# Prepare text chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
texts = text_splitter.split_text(full_text)

# Initialize embeddings
embeddings = OpenAIEmbeddings()

# Create a Chroma database
db = Chroma.from_texts(
    texts=texts,
    embedding=embeddings
)

# Use the database as a retriever
retriever = db.as_retriever()

# Verify retriever functionality
print(retriever)


Created a chunk of size 1216, which is longer than the specified 1000
C:\Users\User\AppData\Local\Temp\ipykernel_15800\3931006293.py:10: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


tags=['Chroma', 'OpenAIEmbeddings'] vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x0000018FFF566910> search_kwargs={}


In [12]:
embeddings = OpenAIEmbeddings()

Initilizing the chat model and multiQuery retriever And outputing the number of unique Documents

In [14]:
from langchain.chat_models import ChatOpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.vectorstores import Chroma

# Initialize ChatOpenAI
llm = ChatOpenAI(temperature=0)

# Ensure the retriever is correctly set up
# Assuming 'retriever' is from a properly initialized Chroma instance

# Create MultiQueryRetriever
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=retriever,  # Existing retriever
    llm=llm
)

# Query the retriever
question = "Who is the little girl?"
unique_docs = retriever_from_llm.get_relevant_documents(query=question)

# Output the number of unique documents
print("Number of unique documents:", len(unique_docs))

# Optionally, display the content of the documents
for i, doc in enumerate(unique_docs[:5]):  # Display the first 5 documents
    print(f"Document {i+1}:\n{doc.page_content}\n")


C:\Users\User\AppData\Local\Temp\ipykernel_15800\1420078782.py:19: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  unique_docs = retriever_from_llm.get_relevant_documents(query=question)


Number of unique documents: 6
Document 1:
Alice took up the fan and gloves, and, as the hall was very
hot, she kept fanning herself all the time she went on talking:
`Dear, dear!  How queer everything is to-day!  And yesterday
things went on just as usual.  I wonder if I've been changed in
the night?  Let me think:  was I the same when I got up this
morning?  I almost think I can remember feeling a little
different.  But if I'm not the same, the next question is, Who in
the world am I?  Ah, THAT'S the great puzzle!'  And she began
thinking over all the children she knew that were of the same age
as herself, to see if she could have been changed for any of
them.

Document 2:
First, she dreamed of little Alice herself, and once again the
tiny hands were clasped upon her knee, and the bright eager eyes
were looking up into hers--she could hear the very tones of her
voice, and see that queer little toss of her head to keep back
the wandering hair that WOULD always get into her eyes--and
st